In [42]:
import pandas as pd
from geopy.distance import great_circle
import zipfile
import os
from shapely.geometry import Point
import xml.etree.ElementTree as ET

In [38]:
sheet_1 = pd.read_excel('расписание_и_причалы.xlsx', sheet_name='Причалы')
sheet_1

,ID причала,Название причала,Адрес причала,Река,координаты точки причала,координаты точки причала.1
0,Docs.id,Docs.name,Docs.address,Docs.river,Docs.latitude,Docs.longitude
1,31,Парк Горького,Пушкинская набережная,Москва,37.597184,55.73013
2,32,Нескучный сад,"г. Москва, Вверх по течению о.Москва от Крымск...",Москва,37.590694,55.722427
3,33,Крымский мост,Фрунзенская набережная напротив д.10,Москва,37.596061,55.732427
4,39,Марьино,"Батайский проезд, д.33",Москва,37.725065,55.641785
5,41,Печатники,"ул.Гурьянова, д.35",Москва,37.714149,55.683699
6,42,Меловой,"Нагатинская набережная, владение 74",Москва,37.696817,55.69106
7,43,Южный речной вокзал,"г. Москва, левый берег р. Москва 1,7-2,2 км. (...",Москва,37.675846,55.689415
8,44,Кленовый бульвар,"г. Москва, Нагатинвская набережная, напротив д...",Москва,37.67215,55.686457
9,45,Воробьевы горы,ул.Косыгина 28с23,Москва,37.546826,55.711738


In [36]:
sheet_1.columns = sheet_1.iloc[0]
sheet_1 = sheet_1.drop(0)
sheet_1.index = sheet_1.index-1
sheet_1

,Docs.id,Docs.name,Docs.address,Docs.river,Docs.latitude,Docs.longitude
0,31,Парк Горького,Пушкинская набережная,Москва,37.597184,55.73013
1,32,Нескучный сад,"г. Москва, Вверх по течению о.Москва от Крымск...",Москва,37.590694,55.722427
2,33,Крымский мост,Фрунзенская набережная напротив д.10,Москва,37.596061,55.732427
3,39,Марьино,"Батайский проезд, д.33",Москва,37.725065,55.641785
4,41,Печатники,"ул.Гурьянова, д.35",Москва,37.714149,55.683699
5,42,Меловой,"Нагатинская набережная, владение 74",Москва,37.696817,55.69106
6,43,Южный речной вокзал,"г. Москва, левый берег р. Москва 1,7-2,2 км. (...",Москва,37.675846,55.689415
7,44,Кленовый бульвар,"г. Москва, Нагатинвская набережная, напротив д...",Москва,37.67215,55.686457
8,45,Воробьевы горы,ул.Косыгина 28с23,Москва,37.546826,55.711738
9,46,Киевский,"г. Москва, Бережковская, между Бородинским и К...",Москва,37.571839,55.743672


In [37]:
sheet_1.to_excel('berths.xlsx', index=True)
sheet_1

,Docs.id,Docs.name,Docs.address,Docs.river,Docs.latitude,Docs.longitude
0,31,Парк Горького,Пушкинская набережная,Москва,37.597184,55.73013
1,32,Нескучный сад,"г. Москва, Вверх по течению о.Москва от Крымск...",Москва,37.590694,55.722427
2,33,Крымский мост,Фрунзенская набережная напротив д.10,Москва,37.596061,55.732427
3,39,Марьино,"Батайский проезд, д.33",Москва,37.725065,55.641785
4,41,Печатники,"ул.Гурьянова, д.35",Москва,37.714149,55.683699
5,42,Меловой,"Нагатинская набережная, владение 74",Москва,37.696817,55.69106
6,43,Южный речной вокзал,"г. Москва, левый берег р. Москва 1,7-2,2 км. (...",Москва,37.675846,55.689415
7,44,Кленовый бульвар,"г. Москва, Нагатинвская набережная, напротив д...",Москва,37.67215,55.686457
8,45,Воробьевы горы,ул.Косыгина 28с23,Москва,37.546826,55.711738
9,46,Киевский,"г. Москва, Бережковская, между Бородинским и К...",Москва,37.571839,55.743672


In [32]:
sheet = sheet_1.set_index(sheet_1['Docs.id'])
sheet

,Docs.id,Docs.name,Docs.address,Docs.river,Docs.latitude,Docs.longitude
Docs.id,,,,,,
31,31,Парк Горького,Пушкинская набережная,Москва,37.597184,55.73013
32,32,Нескучный сад,"г. Москва, Вверх по течению о.Москва от Крымск...",Москва,37.590694,55.722427
33,33,Крымский мост,Фрунзенская набережная напротив д.10,Москва,37.596061,55.732427
39,39,Марьино,"Батайский проезд, д.33",Москва,37.725065,55.641785
41,41,Печатники,"ул.Гурьянова, д.35",Москва,37.714149,55.683699
42,42,Меловой,"Нагатинская набережная, владение 74",Москва,37.696817,55.69106
43,43,Южный речной вокзал,"г. Москва, левый берег р. Москва 1,7-2,2 км. (...",Москва,37.675846,55.689415
44,44,Кленовый бульвар,"г. Москва, Нагатинвская набережная, напротив д...",Москва,37.67215,55.686457
45,45,Воробьевы горы,ул.Косыгина 28с23,Москва,37.546826,55.711738


In [33]:
cords = sheet.drop(sheet.columns[0:4], axis=1)
cords

,Docs.latitude,Docs.longitude
Docs.id,,
31,37.597184,55.73013
32,37.590694,55.722427
33,37.596061,55.732427
39,37.725065,55.641785
41,37.714149,55.683699
42,37.696817,55.69106
43,37.675846,55.689415
44,37.67215,55.686457
45,37.546826,55.711738


In [ ]:
def extract_kmz(kmz_path, extract_to):
    with zipfile.ZipFile(kmz_path, 'r') as kmz:
        kmz.extractall(extract_to)

kmz_file_path = "2024 хакатон.kmz"
extract_to = 'extracted_data'
extract_kmz(kmz_file_path, extract_to)

In [41]:
def parse_kml(kml_path):
    tree = ET.parse(kml_path)
    root = tree.getroot()
    
    namespace = {'kml': 'http://www.opengis.net/kml/2.2'}
    
    routes = []
    
    for placemark in root.findall('.//kml:Placemark', namespace):
        name = placemark.find('kml:name', namespace).text
        coords = placemark.find('.//kml:coordinates', namespace).text.strip().split()
        route_coords = [(float(coord.split(',')[0]), float(coord.split(',')[1])) for coord in coords]
        
        routes.append({'name': name, 'coordinates': route_coords})
    
    return routes

kml_file_path = os.path.join(extract_to, 'doc.kml') 
routes = parse_kml(kml_file_path)

In [44]:
def find_nearest_pier(pier_coords, route_coords):
    pier_point = Point(pier_coords['Docs.latitude'], pier_coords['Docs.longitude'])
    distances = [(Point(lon, lat).distance(pier_point), (lon, lat)) for lat, lon in route_coords]
    return min(distances)[1]

for index, pier in cords.iterrows():
    nearest_points = [find_nearest_pier(pier, route['coordinates']) for route in routes]
    print(f"{pier.index} nearest points on routes: {nearest_points}")

Index(['Docs.latitude', 'Docs.longitude'], dtype='object', name=0) nearest points on routes: [(55.73013, 37.59718399999998), (55.72242700000002, 37.59069400000001), (55.732427, 37.59606099999998), (55.64117899999999, 37.754396), (55.64178500000001, 37.725065), (55.66957700000001, 37.676691), (55.683699, 37.71414900000001), (55.69105999999999, 37.696817), (55.689415, 37.67584599999999), (55.68650099999999, 37.67232000000001), (55.71173799999999, 37.54682600000001), (55.74367199999999, 37.571839), (55.74512900000001, 37.61062699999999), (55.748745, 37.546796), (55.73028099999998, 37.65339200000001), (55.748462, 37.635886), (55.74397200000001, 37.608138), (55.752864, 37.56446299999999), (55.72677599999998, 37.54273699999997), (55.72642299999999, 37.54561500000001), (55.74450200000001, 37.618414), (55.74438700000001, 37.538187), (55.76047231074583, 37.51220315849589), (55.79192000000001, 37.409289), (55.74861200000002, 37.43539499999999), (55.74094400000003, 37.459444), (55.749486, 37.4755

In [46]:
def calculate_distance(pier1, pier2):
    coords1 = (pier1['Docs.latitude'], pier1['Docs.longitude'])
    coords2 = (pier2['Docs.latitude'], pier2['Docs.longitude'])
    return great_circle(coords1, coords2).kilometers

distance_matrix = pd.DataFrame(index=cords.index, columns=cords.index)

for i in range(len(cords)):
    for j in range(len(cords)):
        distance_matrix.iat[i, j] = calculate_distance(cords.iloc[i], cords.iloc[j])

distance_matrix

Docs.id,31,32,33,39,41,42,43,44,45,46,...,73,75,93,97,146,167,172,187,188,192
Docs.id,,,,,,,,,,,,,,,,,,,,,
31,0.0,0.990653,0.237797,16.207334,13.633104,11.600435,9.453068,9.180203,5.829449,3.060456,...,3.953032,19.820448,17.616116,6.338549,4.062297,20.657068,9.816739,4.414948,12.198708,9.728446
32,0.990653,0.0,1.064156,16.542066,14.144655,12.11923,9.904691,9.595412,4.968046,2.810787,...,4.906293,19.307926,17.130666,5.850063,4.352841,20.597549,9.346393,4.685092,12.696072,9.243129
33,0.237797,1.064156,0.0,16.414361,13.813743,11.780702,9.646345,9.379257,5.770348,2.869857,...,3.985208,19.649962,17.441041,6.172962,4.283638,20.42904,9.642729,4.637321,12.382567,9.557295
39,16.207334,16.542066,16.414361,0.0,3.881305,5.353011,6.892952,7.075845,20.75446,19.255082,...,14.26329,35.751216,33.617977,22.381424,12.192958,36.620317,25.871383,11.857117,4.70306,25.759651
41,13.633104,14.144655,13.813743,3.881305,0.0,2.033121,4.288696,4.676383,18.768626,16.681965,...,11.077265,33.441991,31.246813,19.960042,9.87615,33.542179,23.447621,9.58912,1.458742,23.355761
42,11.600435,12.11923,11.780702,5.353011,2.033121,0.0,2.336361,2.772597,16.777398,14.648848,...,9.107562,31.411701,29.214916,17.928984,7.872768,31.570449,21.415579,7.595098,0.662842,21.32427
43,9.453068,9.904691,9.646345,6.892952,4.288696,2.336361,0.0,0.486495,14.480516,12.513363,...,7.406836,29.212576,27.032279,15.744408,5.598459,29.729559,19.239379,5.304417,2.830178,19.141212
44,9.180203,9.595412,9.379257,7.075845,4.676383,2.772597,0.486495,0.0,14.112228,12.239526,...,7.320947,28.897668,26.725918,15.443056,5.262788,29.552144,18.939493,4.956103,3.224393,18.838076
45,5.829449,4.968046,5.770348,20.75446,18.768626,16.777398,14.480516,14.112228,0.0,3.957185,...,9.750437,15.091644,13.054378,3.125189,8.908635,18.258893,5.768224,9.182533,17.309884,5.60336
